Using the domain selected in the previous question, along with all the available data from the dataset, conduct a new analysis of the data.  

a. Formulate a new research question or hypothesis that can be answered using the data available in the survey. Hint: When formulating your new research question, consider exploring under-examined variables (e.g., demographic characteristics of financial planners, behavioral biases, etc.) or alternative outcome measures. You may also consider interacting client characteristics with planner characteristics to uncover new insights. 

b. Develop a conceptual or theoretical framework to address the question. Back your predictions with work from the literature and knowledge from the field.  

c. For your main new empirical analysis, present your results in a table. Explain your methods (e.g., OLS, Probit, etc) and justify why they are applicable here (e.g Probit for binary data, OLS for continuous, etc). Additionally, the mean and standard deviation of all variables used in your analysis must be presented in one descriptive statistics table.  

d. Interpret your results, explaining the significance of the estimated effects and whether they verify your hypothesis. Discuss possible explanations if the hypothesis is not verified.

## 1.DIR

In [129]:
import pandas as pd

# Load dataset
file_path = "clean_all.csv"
data = pd.read_csv(file_path)

# Calculate Debt-to-Income Ratio (DIR) using imputed income and debt values
data['DIR'] = data['debt_impute'] / data['income_impute']
data['DIR'] = data['DIR'].replace([float('inf'), float('nan')], 0)  # Replace inf and NaN with 0


data[['debt_impute', 'income_impute', 'DIR']].head()


,debt_impute,income_impute,DIR
0,580000,110000,5.272727
1,0,200000,0.000000
2,295000,65000,4.538462
3,30000,150000,0.200000
4,0,150000,0.000000


## Risk level 

In [130]:
# Define refined scores for each type of investment product
refined_investment_scores = {
    'etf': 10,
    'stocks': 9,
    'high return mutual fund': 8.5,
    'segregated fund': 8,
    'mutual fund': 7,
    'rrsp': 6.5,
    'tfsa': 6,
    'ul policy': 5.5,
    'mortgage investment': 5,
    'annuity': 3,
    'long term care insurance': 2.5,
    'gic': 2,
    'bond': 1.5,
    'index linked gic': 1.2,
    'repay debt': 1
}

# Define function to score each investment product based on text
def refined_score_investment(text):
    if pd.isna(text):
        return 0
    for product, score in refined_investment_scores.items():
        if product in text.lower():
            return score
    return 0

# Define scenario and corresponding rate columns
scenario_columns = ['scn1a_answer', 'scn2a_answer', 'scn3a_answer', 'scn4a_answer', 'scn1b_answer', 'scn2b_answer', 'scn3b_answer', 'scn4b_answer']
rate_columns = ['scn1a_apr', 'scn1b_apr', 'scn2a_rate', 'scn2b_rate', 'scn3a_rate', 'scn3b_rate', 'scn4a_segfees', 'scn4b_segfees']

# Apply the scoring function to each scenario column
for col in scenario_columns:
    data[f'{col}_refined_investment_score'] = data[col].apply(refined_score_investment)

# Convert rate columns to numeric and fill any NaN values with 1 to avoid null multiplication
data[rate_columns] = data[rate_columns].apply(pd.to_numeric, errors='coerce').fillna(1)

# Calculate the adjusted score by multiplying each scenario score with its corresponding rate
adjusted_scores = []
for i in range(len(scenario_columns)):
    score_col = f'{scenario_columns[i]}_refined_investment_score'
    rate_col = rate_columns[i]
    adjusted_scores.append(data[score_col] * data[rate_col])

# Sum all adjusted scores to get the final adjusted_total_risk_score
data['adjusted_total_risk_score'] = sum(adjusted_scores)

# Preview the adjusted total risk score
print(data[['adjusted_total_risk_score']].head())


   adjusted_total_risk_score
0                       11.5
1                       32.5
2                       29.5
3                       39.0
4                       19.5


According to all the answers from questionaire "SECTION 2. SHOW THE FOLLOWING TITLE TO RESPONDENTS", we refinined the calculation of the adjusted_total_risk_score by incorporating both the investment product scores and corresponding interest rates or fees for each scenario. This approach aims to produce a more nuanced risk score that reflects not only the type of investment recommended but also the associated rate, which can indicate additional risk elements.

We assign scores to each investment product mentioned in the scenario responses based on their inherent risk levels. For instance, high-risk products like ETFs receive a higher score, while low-risk products like bonds receive a lower score.

By multiplying each scenario’s investment product score with its corresponding rate (such as APR or fees), we create a weighted score. Higher rates typically indicate higher financial exposure or risk, so by treating the rate as a multiplier, we capture this aspect within the overall score.

After calculating each scenario’s adjusted score (product score * rate), we sum all scenarios to yield the adjusted_total_risk_score. This final score now accounts for both the inherent risk of each investment type and the added risk implied by high rates or fees.

## log-dir

In [142]:
import numpy as np

# Classify DIR into low, medium, high categories based on quantiles
data['DIR_category'] = pd.qcut(data['DIR'], q=3, labels=['Low', 'Medium', 'High'])

# Apply log transformation to DIR, handling zeros by adding a small constant
data['log_DIR'] = np.log(data['DIR'] + 1e-5)

# Check the DIR categories and log-transformed DIR
data[['DIR', 'DIR_category', 'log_DIR']].head()


,DIR,DIR_category,log_DIR
0,5.272727,High,1.662550
1,0.000000,Low,-11.512925
2,4.538462,High,1.512590
3,0.200000,Low,-1.609388
4,0.000000,Low,-11.512925


## control variables

In [132]:
# Check if control variables are available
control_variables = ['age', 'educ', 'work_experience']
print("Available columns:", data.columns)

# Display control variables
data[control_variables].head()


Available columns: Index(['respid', 'status', 'language', 'gender', 'age', 'province',
       'license_mutualfunds', 'license_insurance', 'license_securities',
       'educ',
       ...
       'scn2a_answer_refined_investment_score',
       'scn3a_answer_refined_investment_score',
       'scn4a_answer_refined_investment_score',
       'scn1b_answer_refined_investment_score',
       'scn2b_answer_refined_investment_score',
       'scn3b_answer_refined_investment_score',
       'scn4b_answer_refined_investment_score', 'adjusted_total_risk_score',
       'DIR_category', 'log_DIR'],
      dtype='object', length=215)


,age,educ,work_experience
0,26,"University certificate, diploma, degree above ...",2.0
1,47,"Bachelor's degree (e.g. B.A., B.Sc., LL.B.)",20.0
2,40,"Bachelor's degree (e.g. B.A., B.Sc., LL.B.)",1.0
3,41,"Bachelor's degree (e.g. B.A., B.Sc., LL.B.)",16.0
4,65,University certificate or diploma below the ba...,15.0


In [133]:
# Define a mapping for the education levels
education_mapping = {
    'Less than high school diploma or its equivalent': 1,
    'High school diploma or a high school equivalency certificate': 2,
    'Trade certificate or diploma': 3,
    'College, CEGEP or other non-university certificate or diploma (other than trades certificates or diplomas)': 4,
    'University certificate or diploma below the bachelor\'s level': 5,
    'Bachelor\'s degree (e.g. B.A., B.Sc., LL.B.)': 6,
    'University certificate, diploma, degree above the bachelor\'s level': 7
}

# Map the education levels in the data to the numeric values
data['educ_level'] = data['educ'].map(education_mapping)

# Check the transformed education levels
data[['educ', 'educ_level']].head()


,educ,educ_level
0,"University certificate, diploma, degree above ...",7
1,"Bachelor's degree (e.g. B.A., B.Sc., LL.B.)",6
2,"Bachelor's degree (e.g. B.A., B.Sc., LL.B.)",6
3,"Bachelor's degree (e.g. B.A., B.Sc., LL.B.)",6
4,University certificate or diploma below the ba...,5


In [144]:
# Convert DIR_category to dummy variables
X = pd.get_dummies(data['DIR_category'], drop_first=True)  # Drop "Low" as baseline
X['log_DIR'] = data['log_DIR']
X['debt'] = data['debt']
X['age'] = data['age']
X['educ'] = data['educ_level']
X['work_experience'] = data['work_experience']

# Define dependent variable
y = data['adjusted_total_risk_score']

# Check the prepared variables for regression
X.head()


,Medium,High,log_DIR,debt,age,educ,work_experience
0,False,True,1.662550,580000.0,26,7,2.0
1,False,False,-11.512925,0.0,47,6,20.0
2,False,True,1.512590,295000.0,40,6,1.0
3,False,False,-1.609388,30000.0,41,6,16.0
4,False,False,-11.512925,0.0,65,5,15.0


## Logit 

In [141]:
data['binary_risk'] = data['adjusted_total_risk_score'].apply(lambda x: 1 if x >= 28 else 0)
y = data['binary_risk']

X = X.apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(y, errors='coerce')

data_combined = pd.concat([X, y], axis=1).dropna().reset_index(drop=True)

X = data_combined.drop(columns=['binary_risk'])
y = data_combined['binary_risk']
X = X.astype(float)
y = y.astype(int)  
X = sm.add_constant(X)

logit_model = sm.Logit(y, X).fit()
print(logit_model.summary())


Optimization terminated successfully.
         Current function value: 0.560529
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:            binary_risk   No. Observations:                  810
Model:                          Logit   Df Residuals:                      802
Method:                           MLE   Df Model:                            7
Date:                 周六, 09 11月 2024   Pseudo R-squ.:                 0.01147
Time:                        23:00:11   Log-Likelihood:                -454.03
converged:                       True   LL-Null:                       -459.30
Covariance Type:            nonrobust   LLR p-value:                    0.1602
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -1.5408      0.668     -2.308      0.021      -2.849      -0.232
Medium        

The coefficient for log_DIR is -0.0632 with a p-value of 0.097, indicating that this effect is close to being statistically significant. This suggests that, although the data does not provide strong evidence, there may still be a meaningful relationship between an analyst’s debt-to-income ratio and their risk tolerance. This trend could become more pronounced with additional data or could reflect behavioral tendencies in financially leveraged individuals.

The coefficient for debt is very close to zero (6.439e-07), but its p-value of 0.047 suggests that it is statistically significant at the 5% level.Although the effect size appears small, statistical significance implies that absolute debt does indeed have an impact on the risk recommendation.

Debt-to-Income Ratio may dilute the perception of financial burden, as it considers both debt and income. It could be that analysts, consciously or subconsciously, regard absolute debt as a more direct measure of financial risk.

## VIF TEST

In [136]:
import statsmodels.api as sm
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import brier_score_loss
from scipy.stats import chi2

vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("Variance Inflation Factor (VIF):")
print(vif_data)


def hosmer_lemeshow_test(model, X, y, groups=10):
    prob = model.predict(X)
    data = pd.DataFrame({"Observed": y, "Predicted": prob})
    data['Decile'] = pd.qcut(data['Predicted'], groups, labels=False)
    hl_table = data.groupby('Decile').apply(lambda df: pd.Series({
        'Obs_1': df['Observed'].sum(),
        'Obs_0': (1 - df['Observed']).sum(),
        'Pred_1': df['Predicted'].sum(),
        'Pred_0': (1 - df['Predicted']).sum()
    }))
    hl_table['Chi_sq'] = (hl_table['Obs_1'] - hl_table['Pred_1']) ** 2 / hl_table['Pred_1'] + \
                         (hl_table['Obs_0'] - hl_table['Pred_0']) ** 2 / hl_table['Pred_0']
    chi_sq = hl_table['Chi_sq'].sum()
    p_value = 1 - chi2.cdf(chi_sq, groups - 2)
    return chi_sq, p_value

chi_sq, p_value = hosmer_lemeshow_test(logit_model, X, y)
print("Hosmer-Lemeshow Test:")
print(f"Chi-square: {chi_sq}, p-value: {p_value}")


Variance Inflation Factor (VIF):
          Variable        VIF
0            const  62.646212
1           Medium   4.844218
2             High   6.759587
3          log_DIR   5.261611
4             debt   1.842343
5              age   2.251609
6             educ   1.017195
7  work_experience   2.212070
Hosmer-Lemeshow Test:
Chi-square: 5.926804828141834, p-value: 0.6554306642640263



VIF Analysis: The VIF results indicate that there is no significant multicollinearity issue in the model, and the variables can be retained without concern for excessive correlation.

Hosmer-Lemeshow Test: The model performs well in terms of binary logistic regression fit, suggesting that the model's ability to predict risk classification aligns well with the observed data.

## mean and standard deviation of all variables used in the analysis

In [137]:
# Calculate mean and standard deviation of all variables used in the analysis
# Assuming X and y are the variables used in the analysis
variables_summary = pd.DataFrame()
variables_summary['Mean'] = X.mean()
variables_summary['Standard Deviation'] = X.std()
variables_summary.loc['binary_risk', 'Mean'] = y.mean()
variables_summary.loc['binary_risk', 'Standard Deviation'] = y.std()
variables_summary = variables_summary.reset_index().rename(columns={'index': 'Variable'})
print(variables_summary)


          Variable           Mean  Standard Deviation
0            const       1.000000            0.000000
1           Medium       0.330864            0.470815
2             High       0.328395            0.469920
3          log_DIR      -2.735257            5.321379
4             debt  248401.575309       322287.536357
5              age      49.379012           11.993130
6             educ       5.601235            1.323085
7  work_experience      16.240741           10.044650
8      binary_risk       0.237037            0.425528


Summary


Our study aimed to examine how personal financial characteristics of financial analysts influence their risk recommendations. Specifically, we focused on two main variables:

Debt-to-Income Ratio (DIR): Calculated as the ratio of an analyst’s total debt to their income, representing the relative financial pressure they may face.
Absolute Debt Level: The total debt amount, which serves as a measure of the absolute financial burden without considering income.
In addition, we analyzed other demographic and professional factors such as age, education level, and work experience, to understand their potential roles in shaping risk recommendation tendencies.

Our findings suggest that personal financial status, specifically relative financial pressure (DIR) and absolute debt, may impact how financial analysts approach risk recommendations. Higher relative debt (DIR) appears to discourage high-risk recommendations, while higher absolute debt has the opposite effect, encouraging them to take on greater risk. This dual influence highlights the complex ways in which financial circumstances may affect professional decision-making in financial advising.

Name: Haotong Sun 11337172
Generate AI used for coding assistance